In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')
from lib import models, graph, coarsening, utils
import tensorflow as tf
import numpy as np
import time
import scipy.sparse
from scipy.io import *
%matplotlib inline


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', os.path.join('..', 'data', 'mnist'), 'Directory to store data.')

# Load Data

In [3]:
# Number of Nearest Neighbors
k = 100

base_storage_dir = '/home/halwai/coursework/honors/Datasets/PASCAL_VOC/GCN/preprocessing/experiment3/'
temp_filename = base_storage_dir + 'scikit_fc7_k_' + str(k) + '.mat'

if os.path.exists(temp_filename):
    print('File already exists\n No Preprocessing Required')
else:
    print('Do preprocessing --> TODO --> call the fucntion with a specific parameter')
    
data = scipy.io.loadmat( temp_filename)

X_train          = data['X_train']
X_test           = data['X_test']
train_labels     = data['Y_train']
test_labels      = data['Y_test']
Adjacency_test   = data['Adjacency_test']
Adjacency_train  = data['Adjacency_train']

#delete unused variables
del data

File already exists
 No Preprocessing Required


In [5]:
#define laplacians, coarsen adjacency, permutate data points.
train_laplacians = [[] for i in range(FLAGS.coarsening_levels)]
test_laplacians = [[] for i in range(FLAGS.coarsening_levels)]

train_perm ,test_perm = [] ,[]

for i in range(X_train.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_train[0][i], levels=FLAGS.coarsening_levels, self_connections=True)
    train_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(FLAGS.coarsening_levels):
        train_laplacians[j].append(temp_L[j])     

for i in range(X_test.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_test[0][i], levels=FLAGS.coarsening_levels, self_connections=True)
    test_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(FLAGS.coarsening_levels):
        test_laplacians[j].append(temp_L[j])     

#del Adjacency_test, Adjacency_train

0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [6]:
print(type(test_laplacians))
print(len(test_laplacians))
for i in range(4):
    print(len(test_laplacians[i]))
    print(test_laplacians[i][0].shape)

<class 'list'>
4
4952
(112, 112)
4952
(56, 56)
4952
(28, 28)
4952
(14, 14)


In [7]:
print(len(train_perm[0]),type(train_perm))
train_data = np.zeros((X_train.shape[0], len(train_perm[0]), X_train.shape[2]))
test_data = np.zeros((X_test.shape[0], len(test_perm[0]), X_test.shape[2]))
for i in range(train_data.shape[0]):
    if not(i%1000):
        print(i)
    train_data[i,:,:] = coarsening.perm_data_point(X_train[i,:,:], train_perm[i])

for i in range(test_data.shape[0]):
    if not(i%1000):
        print(i)
    test_data[i,:,:] = coarsening.perm_data_point(X_test[i,:,:], test_perm[i])

val_data = test_data.copy()
val_laplacians = test_laplacians.copy()
val_labels = test_labels.copy()

#del X_train, Y_train, X_test, Y_test 
#del perm_train, perm_test

112 <class 'list'>
0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [8]:
print(len(test_laplacians))
#print(test_laplacians[0])
L = []
for i in range(FLAGS.coarsening_levels):
    L.append(train_laplacians[i][0])
L = np.array(L)
print(L)
for i in range(4):
    print(L[i].shape)

4
[ array([[-0.01008181, -0.01001553, -0.01000646, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01001553, -0.01008055, -0.01000037, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01000646, -0.01000037, -0.01008912, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])
 array([[-0.02009671, -0.02001936, -0.02000967, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02001936, -0.02009376, -0.01999353, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02000967, -0.01999353, -0.02009773, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
   

# GCN 

In [34]:
common = {}
common['dir_name']       = 'cifar10/'
common['num_epochs']     = 200
common['batch_size']     = 100
common['decay_steps']    = (train_data.shape[0] + val_data.shape[0]) / common['batch_size']
common['eval_frequency'] = 10 * common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
C = train_labels.shape[1]  # number of classes
model_perf = utils.model_perf()

In [39]:
if True:
    name = 'softmax'
    params = common.copy()
    params['dir_name'] += name
    params['regularization'] = 5e-4
    params['dropout']        = 1
    params['learning_rate']  = 0.02
    params['decay_rate']     = 0.95
    params['momentum']       = 0.9
    params['F']              = []
    params['F_0']            = 20
    params['K']              = []
    params['p']              = []
    params['M']              = [C] 
    params['train_laplacians'] = train_laplacians
    params['test_laplacians'] = val_laplacians
    params['val_laplacians'] = test_laplacians
    model_perf.test(models.cgcnn(L, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: logits (softmax)
    representation: M_1 = 20
    weights: M_0 * M_1 = 112 * 20 = 2240
    biases: M_1 = 20
step 2000 / 10022 (epoch 39.91 / 200):
  learning_rate = 7.17e-03, loss_average = 1.04e+00
5604.0 / 7013 = 0.799087409097
  validation precision: 0.57, recall : 0.74, f_measure: 0.65, mAP 0.93, MAP 0.88
  time: 11s (wall 9s)
step 4000 / 10022 (epoch 79.82 / 200):
  learning_rate = 2.57e-03, loss_average = 9.61e-01
5577.0 / 7013 = 0.795237416227
  validation precision: 0.57, recall : 0.74, f_measure: 0.64, mAP 0.93, MAP 0.87
  time: 22s (wall 18s)
step 6000 / 10022 (epoch 119.74 / 200):
  learning_rate = 9.21e-04, loss_average = 9.23e-01
5566.0 / 7013 = 0.793668900613
  validation precision: 0.57, recall : 0.74, f_measure: 0.64, mAP 0.93, MAP 0.87
  time: 33s (wall 26s)
step 8000 / 10022 (epoch 159.65 / 200):
  learning_rate = 3.30e-04, loss_average = 9.10e-01
5565.0 / 7013 = 0.793526308285
  validation precision: 0.57, recall : 0.74, 

In [38]:
print(np.sum(train_labels),np.sum(test_labels))

7584 7013


In [ ]:
# Common hyper-parameters for networks with one convolutional layer.
common['regularization'] = 0
common['dropout']        = 1
common['learning_rate']  = 0.02
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [10]
common['F_0']             = 20
common['K']              = [100]
common['p']              = [1]
common['M']              = [C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [ ]:
# With 'chebyshev2' and 'b2relu', it corresponds to cgcnn2_2(L[0], F=10, K=20).
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
#    params['filter'] = 'chebyshev2'
#    params['brelu'] = 'b2relu'
    model_perf.test(models.cgcnn(train_laplacians[0], **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [48]:
# Common hyper-parameters for LeNet5-like networks.
common['regularization'] = 5e-4
common['dropout']        = 0.5
common['learning_rate']  = 0.02  # 0.03 in the paper but sgconv_sgconv_fc_softmax has difficulty to converge
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [32, 64]
common['F_0']            = 20
common['K']              = [25, 25]
common['p']              = [2, 2]
common['M']              = [512, C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [ ]:
if True:
    name = 'cgconv_cgconv_fc_softmax'  # 'Chebyshev'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 32 / 2 = 1792
    weights: F_0 * F_1 * K_1 = 20 * 32 * 25 = 16000
    biases: F_1 = 32
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 56 * 64 / 2 = 1792
    weights: F_1 * F_2 * K_2 = 32 * 64 * 25 = 51200
    biases: F_2 = 64
  layer 3: fc1
    representation: M_3 = 512
    weights: M_2 * M_3 = 1792 * 512 = 917504
    biases: M_3 = 512
  layer 4: logits (softmax)
    representation: M_4 = 20
    weights: M_3 * M_4 = 512 * 20 = 10240
    biases: M_4 = 20
0 th conv layer 100 112 112
1 th conv layer 100 56 56
step 2000 / 10022 (epoch 39.91 / 200):
  learning_rate = 7.17e-03, loss_average = 1.62e+00
4982.0 / 7013 = 0.71039498075
  validation precision: 0.46, recall : 0.64, f_measure: 0.54, mAP 0.87, MAP 0.80
  time: 80s (wall 284s)
step 4000 / 10022 (epoch 79.82 / 200):
  learning_rate = 2.57e-03, loss_average = 1.46e+00
4989.0 / 7013 = 0.71139312705
  validation precision: 0.46, rec